In [1]:
# Sentiment Analysis - Phase 1
# Install libraries
!pip3 install numpy
!pip3 install opencv-python
!pip3 install torch
!pip install torchmetrics
!pip install einops

# Download the dataset
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/english_train.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/english_test.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/english_dev.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/image_index_train.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/image_index_test.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/image_index_dev.txtt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/sentiment_train.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/sentiment_test.txt"
!wget "https://raw.githubusercontent.com/XL2248/MSCTD/main/MSCTD_data/ende/sentiment_dev.txt"

!pip install --upgrade --no-cache-dir gdown
!gdown "1GAZgPpTUBSfhne-Tp0GDkvSHuq6EMMbj&export=download"
!unzip "/content/train_ende.zip"

!gdown "1B9ZFmSTqfTMaqJ15nQDrRNLqBvo-B39W&export=download"
!unzip "/content/test.zip"

!gdown "12HM8uVNjFg-HRZ15ADue4oLGFAYQwvTA&export=download"
!unzip "/content/dev.zip"

!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

Streaming output truncated to the last 5000 lines.
  inflating: dev/1064.jpg            
  inflating: dev/1065.jpg            
  inflating: dev/1066.jpg            
  inflating: dev/1067.jpg            
  inflating: dev/1068.jpg            
  inflating: dev/1069.jpg            
  inflating: dev/107.jpg             
  inflating: dev/1070.jpg            
  inflating: dev/1071.jpg            
  inflating: dev/1072.jpg            
  inflating: dev/1073.jpg            
  inflating: dev/1074.jpg            
  inflating: dev/1075.jpg            
  inflating: dev/1076.jpg            
  inflating: dev/1077.jpg            
  inflating: dev/1078.jpg            
  inflating: dev/1079.jpg            
  inflating: dev/108.jpg             
  inflating: dev/1080.jpg            
  inflating: dev/1081.jpg            
  inflating: dev/1082.jpg            
  inflating: dev/1083.jpg            
  inflating: dev/1084.jpg            
  inflating: dev/1085.jpg            
  inflating: dev/1086.jpg            

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import ToTensor
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from google.colab.patches import cv2_imshow
import numpy as np
from io import StringIO
import string
import cv2
from PIL import Image

# Part 1 - Section 1
# In order to convert lexical data into numeric format we assume a vocabulary
def build_vocabulary(text_path):

  tokenizer = get_tokenizer('basic_english')
  with open(text_path) as text_reader:
      text = text_reader.read().split("\n")
      tokenized_text = []
      for sentence in text:
        sentence =  tokenizer(sentence)
        tokenized_text.append(sentence)
  unk_token = '<unk>'
  builded_vocab = build_vocab_from_iterator(tokenized_text, specials = [unk_token])
  builded_vocab.set_default_index(-10)
  return builded_vocab


# This function replaces given punc with char in the text
def replace_all(text, puncs, char):
    for punc in puncs:
      text = text.replace(punc, char)
    return text


# Dataset class
class SentimentDataset(Dataset):
  def __init__(self, text_path, image_path, label_path, rawimage_path):
    
    # Assign modalities path 
    self.text_path = text_path
    self.image_path = image_path
    self.rawimage_path = rawimage_path
    self.label_path = label_path


    # Make a vocabulary for text data
    self.vocab = build_vocabulary(self.text_path)
    self.tokenizer = get_tokenizer('basic_english')

    # Transform data to tensor 
    self.transform = ToTensor()

    # Read the text data, clean from punctuations, and split sentences in a list
    with open(self.text_path) as text_reader:
      full_text = replace_all(text_reader.read(), string.punctuation, "").split("\n")

    # Tokenize sentences and use vocab dictionary to convert to numercial data
    temp_text = []
    for t in full_text:
      temp_text.append(self.vocab(self.tokenizer(t)))
    full_text = temp_text

    # Padding the text data to obtain sentences with same size
    max_len = max([len(t) for t in full_text]) 

    temp_text = []
    for t in full_text:
      t += [-1] * (max_len - len(t))
    
    # full_text = [[[t] for t in text] for text in full_text]
    self.full_text = full_text
    # print(full_text)

    # Read labels from file
    with open(self.label_path) as label_reader:
      self.full_labels = np.loadtxt(label_reader)
    
  def __len__(self):

    # In this dataset we assume each utterance as a sample
    # Get the number of utterances

    self.len = len(self.full_labels)-1
    return self.len

  def __getitem__(self, idx):
    

    image = cv2.imread(self.rawimage_path+ str(int(idx)) + ".jpg")

    image = np.asarray(image, dtype = 'float32')
    new_image = np.zeros((1000 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
    image = np.append(image, new_image, axis = 0)
    new_image = np.zeros((image.shape[0], 2000-image.shape[1], image.shape[2]), dtype='float32')
    image = np.append(image, new_image, axis = 1)
    self.image = image

    self.text = self.full_text[int(idx)]

    self.label = self.full_labels[idx]
    return {'image': self.image, 'text': self.text, 'ID': idx, 'label':self.label}


In [ ]:
# Part 1 - Section 1
# Extract faces for face recognition
# Warning! It may take long time to run (about 2-3 hours)
# You can use the text files instead of run this section

def make_dataset(data_loader, filename):
  face_cascade  = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")

  all_faces = []
  all_labels = []
  all_IDs = []

  i = 0
  for batch in data_loader:
    i += 1
    # if i == 10:
    #   break
    images = batch['image']
    labels = batch['label']
    IDs = batch['ID']
    for image, label, ID in zip(images, labels, IDs):
      faces = face_cascade.detectMultiScale(image.numpy(), 1.1, 4)
      print(ID)
      for face in faces:
        all_faces.append(face)
        all_labels.append(label)
        all_IDs.append(ID)

  np.save(filename+'_face_location.npy', all_faces)
  np.save(filename+'_face_labels.npy', all_labels)
  np.save(filename+'_face_IDs.npy', all_IDs)


text_path = "/content/english_train.txt"
image_path = "/content/image_index_train.txt"
label_path = "/content/sentiment_train.txt"
rawimage_path = "/content/train_ende/"

train_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
train_data_loader = DataLoader(train_dataset, batch_size=10, shuffle = False)
make_dataset(train_data_loader, 'train')

text_path = "/content/english_dev.txt"
image_path = "/content/image_index_dev.txt"
label_path = "/content/sentiment_dev.txt"
rawimage_path = "/content/dev/"

dev_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
dev_data_loader = DataLoader(dev_dataset, batch_size=10, shuffle = False)
make_dataset(dev_data_loader, 'dev')



Streaming output truncated to the last 5000 lines.
tensor(62)
tensor(63)
tensor(64)
tensor(65)
tensor(66)
tensor(67)
tensor(68)
tensor(69)
tensor(70)
tensor(71)
tensor(72)
tensor(73)
tensor(74)
tensor(75)
tensor(76)
tensor(77)
tensor(78)
tensor(79)
tensor(80)
tensor(81)
tensor(82)
tensor(83)
tensor(84)
tensor(85)
tensor(86)
tensor(87)
tensor(88)
tensor(89)
tensor(90)
tensor(91)
tensor(92)
tensor(93)
tensor(94)
tensor(95)
tensor(96)
tensor(97)
tensor(98)
tensor(99)
tensor(100)
tensor(101)
tensor(102)
tensor(103)
tensor(104)
tensor(105)
tensor(106)
tensor(107)
tensor(108)
tensor(109)
tensor(110)
tensor(111)
tensor(112)
tensor(113)
tensor(114)
tensor(115)
tensor(116)
tensor(117)
tensor(118)
tensor(119)
tensor(120)
tensor(121)
tensor(122)
tensor(123)
tensor(124)
tensor(125)
tensor(126)
tensor(127)
tensor(128)
tensor(129)
tensor(130)
tensor(131)
tensor(132)
tensor(133)
tensor(134)
tensor(135)
tensor(136)
tensor(137)
tensor(138)
tensor(139)
tensor(140)
tensor(141)
tensor(142)
tensor(143)
ten

In [ ]:
temp = []
for i in range(1000):
  temp.append(np.zeros((1000, 1000)))

In [2]:
# Part 1 - Section 1
# Make a dataloader for face recognition using text files
# It may take high RAM (managebale with public colab service)

class FaceDataset(Dataset):
  def __init__(self, rawimage_path, facelocations_path, faceIDs_path, facelabels_path):
    self.rawimage_path = rawimage_path
    self.face_locations = np.load(facelocations_path)
    self.face_IDs = np.load(faceIDs_path)
    self.face_labels = np.load(facelabels_path)

    self.all_faces= []

    for idx in range(len(self.face_locations)):

      index = self.face_IDs[idx]
      image = cv2.imread(self.rawimage_path+ str(int(index)) + ".jpg")

      new_image = np.zeros((1000 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 0)
      new_image = np.zeros((image.shape[0], 2000-image.shape[1], image.shape[2]), dtype='float32')
      self.image = np.append(image, new_image, axis = 1)

      self.location = self.face_locations[idx]
      image = self.image[self.location[1]:self.location[1] + self.location[3], self.location[0]:self.location[0] + self.location[2]]

      while image.shape[0]>100 or image.shape[1]>100:
        image = cv2.resize(image, (int(0.9* image.shape[0]), int(0.9*image.shape[1])))

      image = cv2.resize(image, (int(int(100/image.shape[0])* image.shape[0]), (int(100/image.shape[1]) * image.shape[1])))
      
      new_image = np.zeros((100 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 0)
      new_image = np.zeros((image.shape[0], 100-image.shape[1], image.shape[2]), dtype='float32')
      self.image = np.append(image, new_image, axis = 1)

      self.all_faces.append(self.image)

    
  def __len__(self):
    
    self.len = len(self.face_locations)
    return self.len

  def __getitem__(self, idx):
    
    index = self.face_IDs[idx]

    self.label = np.zeros((3), dtype='float32')
    self.label[int(self.face_labels[idx])] = 1
    self.location = self.face_locations[idx]
    self.image = self.all_faces[index]

    return {'image': self.image, 'ID': index, 'label':self.label}


rawimage_path = "/content/train_ende/"
facelocations_path = "/content/train_face_location.npy"
faceIDs_path = "/content/train_face_IDs.npy"
facelabels_path = "/content/train_face_labels.npy"

train_facedataset = FaceDataset(rawimage_path, facelocations_path, faceIDs_path, facelabels_path)
train_face_data_loader = DataLoader(train_facedataset, batch_size=64, shuffle = True)

rawimage_path = "/content/dev/"
facelocations_path = "/content/dev_face_location.npy"
faceIDs_path = "/content/dev_face_IDs.npy"
facelabels_path = "/content/dev_face_labels.npy"

dev_facedataset = FaceDataset(rawimage_path, facelocations_path, faceIDs_path, facelabels_path)
dev_face_data_loader = DataLoader(dev_facedataset, batch_size=64, shuffle = True)



In [15]:
from torch.nn import Softmax
class CNN(torch.nn.Module):
	def __init__(self):
		super().__init__()
		self.model = torch.nn.Sequential(
			
			torch.nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			torch.nn.MaxPool2d(kernel_size=2),

			torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			torch.nn.MaxPool2d(kernel_size=2),
			
			torch.nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			torch.nn.MaxPool2d(kernel_size=2),

			torch.nn.Flatten(),
			torch.nn.Linear(4608 , 512),
			torch.nn.ReLU(),
      torch.nn.Linear(512, 512),
			torch.nn.ReLU(),
      torch.nn.Linear(512, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 3)
		)

	def forward(self, x):
		return self.model(x)


In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(torch.cuda.is_available())
face_model = CNN().to(device)

epoch_num = 10
lr = 0.00001

cross_entropy = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(face_model.parameters(), lr = lr)

train_losses = []
dev_losses = []

for epoch in range(epoch_num):

  print("epoch ", epoch, "/", epoch_num, end = ": ")
  train_loss = 0
  dev_loss = 0

  face_model.train()
  
  for i, batch in enumerate(train_face_data_loader):
    
    images = torch.permute(batch['image'], (0, 3, 2, 1)).to(device)
    labels =  batch['label'].to(device)

    outputs = face_model(images)
    loss = cross_entropy(outputs, labels)
  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  train_losses.append(train_loss/len(train_face_data_loader))
  torch.save(face_model.state_dict(), "model1_" + str(epoch))

  for i, batch in enumerate(dev_face_data_loader):

    images = torch.permute(batch['image'].to(device), (0, 3, 2, 1))
    labels =  batch['label'].to(device)

    outputs = face_model(images)
    loss = cross_entropy(outputs, labels)
    dev_loss += loss.item()

  dev_losses.append(dev_loss/len(dev_face_data_loader))

  print("\ntrain loss: ", train_losses[-1], ", dev loss: ", dev_losses[-1])




True
epoch  0 / 10: 
train loss:  1.0885353955355557 , dev loss:  1.0958716411783238
epoch  1 / 10: 
train loss:  1.0751192490436021 , dev loss:  1.0986629278972895
epoch  2 / 10: 
train loss:  1.0588769054994351 , dev loss:  1.111206783188714
epoch  3 / 10: 
train loss:  1.0344284760978428 , dev loss:  1.1280453650638311
epoch  4 / 10: 
train loss:  1.0052956823763457 , dev loss:  1.131905416045526
epoch  5 / 10: 
train loss:  0.9650839957059619 , dev loss:  1.1579701575365933
epoch  6 / 10: 
train loss:  0.9245501345647148 , dev loss:  1.1942261770518139
epoch  7 / 10: 
train loss:  0.8798820451727992 , dev loss:  1.215545826488071
epoch  8 / 10: 
train loss:  0.8248691264118694 , dev loss:  1.2587757158761073
epoch  9 / 10: 
train loss:  0.7789380685188818 , dev loss:  1.2722555206279562


In [47]:
index = np.argmin(dev_losses)
face_model = CNN()
face_model.load_state_dict(torch.load("model1_" + str(index)))
face_model.eval()

CNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=4608, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=3, bias=True)
  )
)

In [48]:
# This function Extract face by given image
def extract_face(image):

  face_cascade  = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")
  image = np.asarray(image, dtype = 'uint8')
  face_locations = face_cascade.detectMultiScale(image, 1.1, 4)

  face_images = []
  for location in face_locations:

    face = np.asarray(image[location[1]:location[1] + location[3], location[0]:location[0] + location[2]])


    while face.shape[0]>100 or face.shape[1]>100:
      face = cv2.resize(face, (int(0.9* face.shape[0]), int(0.9*face.shape[1])))

    face = cv2.resize(face, (int(int(100/face.shape[0])* face.shape[0]), (int(100/face.shape[1]) * face.shape[1])))

    new_face = np.zeros((100 - face.shape[0], face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 0)
    new_face = np.zeros((face.shape[0], 100-face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 1)

    face_images.append(face)

  return np.asarray(face_images, dtype = 'float32')


# Create the test dataset 
text_path = "/content/english_test.txt"
image_path = "/content/image_index_test.txt"
label_path = "/content/sentiment_test.txt"
rawimage_path = "/content/test/"

test_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle = False)

# Evaluate the Model by test dataset

outputs = []
true_output = []
i = 0
with torch.no_grad():
  for batch in test_data_loader:
    images = batch['image']
    labels = batch['label']
    # if i == 100:
    #   break
    i += 1
    for image, label in zip(images, labels): 
      temp_output = []
      for face in extract_face(image):

        face = np.asarray([face])
        output = face_model(torch.permute(torch.Tensor(face), (0, 3, 2, 1)))
        output = torch.softmax(output, dim = 1)
        temp_output.append(torch.argmax(output).to('cpu'))

      if len(temp_output)==0:
        temp_output.append(1)

      u, c = np.unique(np.asarray(temp_output), return_counts = True)

      outputs += [u[np.argmax(c)]]
      true_output.append(label)

from torchmetrics import Accuracy
true_output = torch.tensor(true_output)
outputs = torch.tensor(outputs)
Accuracy(task="multiclass", num_classes=3)(true_output, outputs)


tensor(0.3731)

In [49]:
import math

import torch
from einops import parse_shape, rearrange
from opt_einsum import contract
    
import matplotlib.pyplot as plt
import PIL.Image
import torchvision.transforms as T

class RandomSmoothColor(torch.nn.Module):
    def __init__(self, cut, T, freq_bandwidth=None, stochastic=False, T_min=0.):
        super().__init__()
        self.cut = cut
        self.T = T
        self.freq_bandwidth = freq_bandwidth
        
        self.stochastic = stochastic
        if self.stochastic:
            self.cut_max = cut
            self.T_min = T_min
            self.T_max = T

    def forward(self, img):

        if self.stochastic:
            self._sample_params()

        init_shape = img.shape
        if len(init_shape) < 4:
            img = rearrange(img, "c h w -> () c h w")

        return self.random_smooth_color(img, self.cut, self.T, self.freq_bandwidth).reshape(init_shape)

    def _sample_params(self):
        self.cut = torch.randint(low=1, high=self.cut_max + 1, size=(1,)).item()
        self.T = torch.FloatTensor([1]).uniform_(self.T_min, self.T_max).item()

    def random_smooth_color(self, img, cut, T, freq_bandwidth=None):
        img_shape = parse_shape(img, "b c h w")
        colors = rearrange(img, "b c h w -> b c (h w)")
        if freq_bandwidth is not None:
            min_k = torch.randint(low=1, high=cut + 1, size=(1,)).item()
            k = torch.arange(
                min_k, min(min_k + freq_bandwidth, cut + 1), 
                device=img.device
            )
            coeff = torch.randn(
                (img_shape["b"], img_shape["c"], k.shape[0]), 
                device=img.device
            )
        else:
            coeff = torch.randn(
                (img_shape["b"], img_shape["c"], cut), 
                device=img.device
            )
            k = torch.arange(1, cut + 1, device=img.device)

        coeff = coeff * torch.sqrt(torch.tensor(T))

        freqs = torch.sin(colors[..., None] * k[None, None, None, :] * math.pi)

        # transformed_colors = torch.einsum("bcf,bcnf->bcn", coeff, freqs) + colors
        transformed_colors = contract("bcf, bcnf -> bcn", coeff, freqs) + colors
        transformed_colors = torch.clamp(transformed_colors, 0, 1)

        transformed_image = rearrange(transformed_colors, " b c (h w) -> b c h w", **img_shape)
        return transformed_image

    def __repr__(self):
        return self.__class__.__name__ + f"(T={self.T}, cut={self.cut})"


def filter_2(image):


        random_color = RandomSmoothColor(100, 0.02)
        x = T.ToTensor()(image).to('cuda')
        x2 = random_color(x)
        t = 0
        return x2.permute(1, 2, 0)

In [50]:
import numpy as np
import torch
from einops import parse_shape, rearrange
import matplotlib.pyplot as plt
import PIL.Image
import torchvision.transforms as T


class RandomFilter(torch.nn.Module):
    def __init__(self, kernel_size, sigma, stochastic=False, sigma_min=0.):
        super().__init__()
        self.kernel_size = kernel_size
        self.sigma = sigma

        self.stochastic = stochastic
        if self.stochastic:
            self.kernels_size_candidates = torch.tensor([float(i) for i in range(self.kernel_size, self.kernel_size + 2, 2)])
            self.sigma_min = sigma_min
            self.sigma_max = sigma

    def forward(self, img):
        if self.stochastic:
            self._sample_params()

        init_shape = img.shape
        if len(init_shape) < 4:
            img = rearrange(img, "c h w -> () c h w")

        shape_dict = parse_shape(img, "b c h w")
        batch_size = shape_dict["b"]
        img = rearrange(img, "b c h w -> c b h w")

        delta = torch.zeros((1, self.kernel_size, self.kernel_size), device=img.device)
        center = int(np.ceil(self.kernel_size / 2))
        delta[0, center, center] = 1.0

        conv_weight = rearrange(
            self.sigma * torch.randn((batch_size, self.kernel_size, self.kernel_size), device=img.device) + delta,
            "b h w -> b (h w)",
        )

        conv_weight = rearrange(conv_weight, "b (h w) -> b () h w", h=self.kernel_size)

        filtered_img = torch.nn.functional.conv2d(
            img, conv_weight, padding="same", groups=batch_size
        )

        # Deal with NaN values due to mixed precision -> Convert them to 1.
        filtered_img[filtered_img.isnan()] = 1.

        filtered_img = rearrange(filtered_img, "c b h w -> b c h w")
        filtered_img = torch.clamp(filtered_img, 0., 1.).reshape(init_shape)

        return filtered_img

    def _sample_params(self):
        self.kernel_size = int(self.kernels_size_candidates[torch.multinomial(self.kernels_size_candidates, 1)].item())
        self.sigma = torch.FloatTensor([1]).uniform_(self.sigma_min, self.sigma_max).item()

    def __repr__(self):
        return self.__class__.__name__ + f"(sigma={self.sigma}, kernel_size={self.kernel_size})"


def filter_1(image):

        random_filter = RandomFilter(3, 1)
        x = T.ToTensor()(image).to('cuda')
        x2 = random_filter.forward(x)
        
        t = 0
        return x2.permute(1, 2, 0)

In [51]:
import functools
import math
import torch
from einops import rearrange
from opt_einsum import contract


class Diffeo(torch.nn.Module):
    """Randomly apply a diffeomorphism to the image(s).
    The image should be a Tensor and it is expected to have [..., n, n] shape,
    where ... means an arbitrary number of leading dimensions.
    A random cut is drawn from a discrete Beta distribution of parameters
    alpha and beta such that
        s = alpha + beta (measures how peaked the distribution is)
        r = alpha / beta (measured how biased towards cutmax the distribution is)
    Given cut and the allowed* interval of temperatures [Tmin, Tmax], a random T is
    drawn from a Beta distribution with parameters alpha and beta such that:
        s = alpha + beta (measures how peaked the distribution is)
        r = alpha / beta (measured how biased towards T_max the distribution is)
    Beta ~ delta_function for s -> inf. To apply a specific value x \in [0, 1]
    in the allowed interval of T or cut, set
        - s = 1e10
        - r = x / (1 - x)
    *the allowed T interval is defined such as:
        - Tmin corresponds to a typical displacement of 1/2 pixel in the center
          of the image
        - Tmax corresponds to the highest T for which no overhangs are present.
    Args:
        sT (float):
        rT (float):
        scut (float):
        rcut (float):
        cut_min (int):
        cut_max (int):
    Returns:
        Tensor: Diffeo version of the input image(s).
    """

    def __init__(self, sT, rT, scut, rcut, cutmin, cutmax, alpha, stochastic=False):
        super().__init__()

        self.sT = sT
        self.rT = rT
        self.scut = scut
        self.rcut = rcut
        self.cutmin = cutmin
        self.cutmax = cutmax
        self.alpha = alpha

        self.stochastic = stochastic
        if self.stochastic:
            self.cutmax_max = cutmax
            self.alpha_max = alpha

        self.betaT = torch.distributions.beta.Beta(sT - sT / (rT + 1), sT / (rT + 1), validate_args=None)
        self.betacut = torch.distributions.beta.Beta(scut - scut / (rcut + 1), scut / (rcut + 1), validate_args=None)

    def forward(self, img):
        """
        Args:
            img (Tensor): Image(s) to be 'diffeomorphed'.
        Returns:
            Tensor: Diffeo image(s).
        """

        init_shape = img.shape
        if len(init_shape) < 4:
            img = rearrange(img, "c h w -> () c h w")

        if self.stochastic:
            self._sample_params()

        # image size
        n = img.shape[-1]

        cut = (self.betacut.sample() * (self.cutmax + 1 - self.cutmin) + self.cutmin).int().item()
        T1, T2 = temperature_range(n, cut)
        T2 = max(T1, self.alpha * T2)
        T = (self.betaT.sample() * (T2 - T1) + T1)

        return deform(img, T, cut).reshape(init_shape)

    def _sample_params(self):
        self.cutmax = torch.randint(low=self.cutmin + 1, high=self.cutmax_max + 1, size=(1,)).item()
        # self.alpha = torch.FloatTensor([1]).uniform_(0., self.alpha_max).item()

    def __repr__(self):
        return self.__class__.__name__ + f'(sT={self.sT}, rT={self.rT}, scut={self.scut}, rcut={self.rcut}, cutmin={self.cutmin}, cutmax={self.cutmax})'


def scalar_field_modes(n, m, dtype=torch.float64, device='cpu'):
    """
    sqrt(1 / Energy per mode) and the modes
    """
    x = torch.linspace(0, 1, n, dtype=dtype, device=device)
    k = torch.arange(1, m + 1, dtype=dtype, device=device)
    i, j = torch.meshgrid(k, k)
    r = (i.pow(2) + j.pow(2)).sqrt()
    e = (r < m + 0.5) / r
    s = torch.sin(math.pi * x[:, None] * k[None, :])
    return e, s


def scalar_field(n, m, device='cpu'):
    """
    random scalar field of size nxn made of the first m modes
    """
    e, s = scalar_field_modes(n, m, dtype=torch.get_default_dtype(), device=device)
    c = torch.randn(m, m, device=device) * e
    # return torch.einsum('ij,xi,yj->yx', c, s, s)
    return contract('ij,xi,yj->yx', c, s, s)


def deform(image, T, cut, interp='linear'):
    """
    1. Sample a displacement field tau: R2 -> R2, using tempertature `T` and cutoff `cut`
    2. Apply tau to `image`
    :param img Tensor: square image(s) [..., y, x]
    :param T float: temperature
    :param cut int: high frequency cutoff
    """
    n = image.shape[-1]
    assert image.shape[-2] == n, 'Image(s) should be square.'

    device = image.device

    # Sample dx, dy
    # u, v are defined in [0, 1]^2
    # dx, dx are defined in [0, n]^2
    u = scalar_field(n, cut, device)  # [n,n]
    v = scalar_field(n, cut, device)  # [n,n]
    dx = T ** 0.5 * u * n
    dy = T ** 0.5 * v * n

    # Apply tau
    return remap(image, dx, dy, interp)


def remap(a, dx, dy, interp):
    """
    :param a: Tensor of shape [..., y, x]
    :param dx: Tensor of shape [y, x]
    :param dy: Tensor of shape [y, x]
    :param interp: interpolation method
    """
    n, m = a.shape[-2:]
    assert dx.shape == (n, m) and dy.shape == (n, m), 'Image(s) and displacement fields shapes should match.'

    y, x = torch.meshgrid(torch.arange(n, dtype=dx.dtype, device=a.device), torch.arange(m, dtype=dx.dtype, device=a.device))

    xn = (x - dx).clamp(0, m-1)
    yn = (y - dy).clamp(0, n-1)

    if interp == 'linear':
        xf = xn.floor().long()
        yf = yn.floor().long()
        xc = xn.ceil().long()
        yc = yn.ceil().long()

        xv = xn - xf
        yv = yn - yf

        return (1-yv)*(1-xv)*a[..., yf, xf] + (1-yv)*xv*a[..., yf, xc] + yv*(1-xv)*a[..., yc, xf] + yv*xv*a[..., yc, xc]

    if interp == 'gaussian':
        # can be implemented more efficiently by adding a cutoff to the Gaussian
        sigma = 0.4715

        dx = (xn[:, :, None, None] - x)
        dy = (yn[:, :, None, None] - y)

        c = (-dx**2 - dy**2).div(2 * sigma**2).exp()
        c = c / c.sum([2, 3], keepdim=True)

        return (c * a[..., None, None, :, :]).sum([-1, -2])


def temperature_range(n, cut):
    """
    Define the range of allowed temperature
    for given image size and cut.
    """
    if cut == 0:
        print("Cut is zero!")
    if isinstance(cut, (float, int)):
        cut = cut + 1e-6
        log = math.log(cut)
    else:
        log = cut.log()
    T1 = 1 / (math.pi * n ** 2 * log)
    T2 = 4 / (math.pi ** 3 * cut ** 2 * log)
    return T1, T2


def typical_displacement(T, cut, n):
    if isinstance(cut, (float, int)):
        log = math.log(cut)
    else:
        log = cut.log()
    return n * (math.pi * T * log) ** .5 / 2

def filter_3(image):
  image = T.ToTensor()(image).to('cuda')
  image = Diffeo(1, 1, 1, 1, 2, 100, 1.0)(image).permute(1,2, 0)
  return image
  

In [52]:
# Part 1 - Section 2

# Boost dataset by given transformers (see filter_1, filter_2, filter_3)
# Make a dataloader for face recognition using text files
# It may take high RAM (managebale with public colab service)

class FaceDataset(Dataset):
  def __init__(self, rawimage_path, facelocations_path, faceIDs_path, facelabels_path):
    self.rawimage_path = rawimage_path
    self.face_locations = np.load(facelocations_path)
    self.face_IDs = np.load(faceIDs_path)
    self.face_labels = np.load(facelabels_path)

    self.all_faces= []

    for idx in range(len(self.face_locations)):

      index = self.face_IDs[idx]
      image = cv2.imread(self.rawimage_path+ str(int(index)) + ".jpg")

      new_image = np.zeros((1000 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 0)
      new_image = np.zeros((image.shape[0], 2000-image.shape[1], image.shape[2]), dtype='float32')
      self.image = np.append(image, new_image, axis = 1)

      self.location = self.face_locations[idx]
      image = self.image[self.location[1]:self.location[1] + self.location[3], self.location[0]:self.location[0] + self.location[2]]

      while image.shape[0]>100 or image.shape[1]>100:
        image = cv2.resize(image, (int(0.9* image.shape[0]), int(0.9*image.shape[1])))

      image = cv2.resize(image, (int(int(100/image.shape[0])* image.shape[0]), (int(100/image.shape[1]) * image.shape[1])))
      
      new_image = np.zeros((100 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 0)
      new_image = np.zeros((image.shape[0], 100-image.shape[1], image.shape[2]), dtype='float32')
      self.image = np.append(image, new_image, axis = 1)

      self.all_faces.append(self.image)

    self.original_len = len(self.face_locations)
    
  def __len__(self):
    
    self.len = 4*len(self.face_locations)
    
    return self.len

  def __getitem__(self, idx):

    case_num = int(idx/(self.original_len))
    idx = idx%(self.original_len)

    index = self.face_IDs[idx]

    self.label = np.zeros((3), dtype='float32')
    self.label[int(self.face_labels[idx])] = 1
    self.location = self.face_locations[idx]
    self.image = self.all_faces[index]
    
    if case_num == 1 :
      self.image = filter_1(self.image)

    elif case_num == 2:
      self.image = filter_2(self.image)

    elif case_num == 3:
      self.image = filter_3(self.image)
      

    else:

      self.image = T.ToTensor()(self.image).to('cuda')
      self.image = torch.permute(self.image, (2, 1, 0))
  
    return {'image': self.image, 'ID': index, 'label':self.label}


rawimage_path = "/content/train_ende/"
facelocations_path = "/content/train_face_location.npy"
faceIDs_path = "/content/train_face_IDs.npy"
facelabels_path = "/content/train_face_labels.npy"

train_facedataset = FaceDataset(rawimage_path, facelocations_path, faceIDs_path, facelabels_path)
train_face_data_loader = DataLoader(train_facedataset, batch_size=64, shuffle = True)

rawimage_path = "/content/dev/"
facelocations_path = "/content/dev_face_location.npy"
faceIDs_path = "/content/dev_face_IDs.npy"
facelabels_path = "/content/dev_face_labels.npy"

dev_facedataset = FaceDataset(rawimage_path, facelocations_path, faceIDs_path, facelabels_path)
dev_face_data_loader = DataLoader(dev_facedataset, batch_size=64, shuffle = True)




In [54]:
# Train model again by boosted dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(torch.cuda.is_available())
face_model = CNN().to(device)

epoch_num = 8
lr = 0.00001

cross_entropy = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(face_model.parameters(), lr = lr)

train_losses = []
dev_losses = []

for epoch in range(epoch_num):

  print("epoch ", epoch, "/", epoch_num, end = ": ")
  train_loss = 0
  dev_loss = 0

  face_model.train()
  
  for i, batch in enumerate(train_face_data_loader):
    # print(i, end = ", ")
    
    images = batch['image'].to(device)

    images = torch.permute(images, (0, 3, 2, 1))
    labels =  batch['label'].to(device)
    
    outputs = face_model(images)
    loss = cross_entropy(outputs, labels)
  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  train_losses.append(train_loss/len(train_face_data_loader))
  torch.save(face_model.state_dict(), "model2_" + str(epoch))

  for i, batch in enumerate(dev_face_data_loader):

    images = torch.permute(batch['image'].to(device), (0, 3, 2, 1))
    labels =  batch['label'].to(device)

    outputs = face_model(images)
    loss = cross_entropy(outputs, labels)
    dev_loss += loss.item()

  dev_losses.append(dev_loss/len(dev_face_data_loader))

  print("\ntrain loss: ", train_losses[-1], ", dev loss: ", dev_losses[-1])




True
epoch  0 / 8: 
train loss:  1.0901645647566538 , dev loss:  1.096824876869781
epoch  1 / 8: 
train loss:  1.0764098345373367 , dev loss:  1.1039918706386904
epoch  2 / 8: 
train loss:  1.0570928752091564 , dev loss:  1.1171428330336945
epoch  3 / 8: 
train loss:  1.031793589200349 , dev loss:  1.1269295245786257
epoch  4 / 8: 
train loss:  1.0004200623448232 , dev loss:  1.1469575471515896
epoch  5 / 8: 
train loss:  0.9668559998968466 , dev loss:  1.1730228283737278
epoch  6 / 8: 
train loss:  0.9335793631521366 , dev loss:  1.1870641258698476
epoch  7 / 8: 
train loss:  0.9000254025337566 , dev loss:  1.2561589319494706


In [56]:
index = np.argmin(dev_losses)
face_model = CNN()
face_model.load_state_dict(torch.load("model2_" + str(index)))
face_model.eval()

CNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=4608, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=3, bias=True)
  )
)

In [58]:
def extract_face(image):

  face_cascade  = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")
  image = np.asarray(image, dtype = 'uint8')
  face_locations = face_cascade.detectMultiScale(image, 1.1, 4)

  face_images = []
  for location in face_locations:

    face = np.asarray(image[location[1]:location[1] + location[3], location[0]:location[0] + location[2]])


    while face.shape[0]>100 or face.shape[1]>100:
      face = cv2.resize(face, (int(0.5* face.shape[0]), int(0.5*face.shape[1])))

    face = cv2.resize(face, (int(int(100/face.shape[0])* face.shape[0]), (int(100/face.shape[1]) * face.shape[1])))

    new_face = np.zeros((100 - face.shape[0], face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 0)
    new_face = np.zeros((face.shape[0], 100-face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 1)

    face_images.append(face)

  return np.asarray(face_images, dtype = 'float32')


# Create the test dataset 
text_path = "/content/english_test.txt"
image_path = "/content/image_index_test.txt"
label_path = "/content/sentiment_test.txt"
rawimage_path = "/content/test/"

test_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle = False)

outputs = []
true_output = []
i = 0
with torch.no_grad():
  for batch in test_data_loader:
    images = batch['image']
    labels = batch['label']
    # if i == 10:
    #   break
    i += 1
    for image, label in zip(images, labels): 
        temp_output = []
        for face in extract_face(image):

          face = np.asarray([face])
          output = face_model(torch.permute(torch.Tensor(face), (0, 3, 2, 1)))
          output = torch.softmax(output, dim = 1)
          temp_output.append(torch.argmax(output).to('cpu'))

        if len(temp_output)==0:
          temp_output.append(1)

        u, c = np.unique(np.asarray(temp_output), return_counts = True)

        outputs += [u[np.argmax(c)]]
        true_output.append(label)
        
from torchmetrics import Accuracy
true_output = torch.tensor(true_output)
outputs = torch.tensor(outputs)
Accuracy(task="multiclass", num_classes=3)(true_output, outputs)


tensor(0.3926)

In [5]:
from torchvision.models import densenet121, DenseNet121_Weights

densenet = densenet121(weights= DenseNet121_Weights.IMAGENET1K_V1)

In [60]:
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import ToTensor
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from google.colab.patches import cv2_imshow
import numpy as np
from io import StringIO
import string
import cv2
from PIL import Image

# Part 1
# In order to convert lexical data into numeric format we assume a vocabulary
def build_vocabulary(text_path):

  tokenizer = get_tokenizer('basic_english')
  with open(text_path) as text_reader:
      text = text_reader.read().split("\n")
      tokenized_text = []
      for sentence in text:
        sentence =  tokenizer(sentence)
        tokenized_text.append(sentence)
  unk_token = '<unk>'
  builded_vocab = build_vocab_from_iterator(tokenized_text, specials = [unk_token])
  builded_vocab.set_default_index(-10)
  return builded_vocab


# This function replaces given punc with char in the text
def replace_all(text, puncs, char):
    for punc in puncs:
      text = text.replace(punc, char)
    return text


# Dataset class
class SentimentDataset(Dataset):
  def __init__(self, text_path, image_path, label_path, rawimage_path):
    
    # Assign modalities path 
    self.text_path = text_path
    self.image_path = image_path
    self.rawimage_path = rawimage_path
    self.label_path = label_path


    # Make a vocabulary for text data
    self.vocab = build_vocabulary(self.text_path)
    self.tokenizer = get_tokenizer('basic_english')

    # Transform data to tensor 
    self.transform = ToTensor()

    # Read the text data, clean from punctuations, and split sentences in a list
    with open(self.text_path) as text_reader:
      full_text = replace_all(text_reader.read(), string.punctuation, "").split("\n")

    # Tokenize sentences and use vocab dictionary to convert to numercial data
    temp_text = []
    for t in full_text:
      temp_text.append(self.vocab(self.tokenizer(t)))
    full_text = temp_text

    # Padding the text data to obtain sentences with same size
    max_len = max([len(t) for t in full_text]) 

    temp_text = []
    for t in full_text:
      t += [-1] * (max_len - len(t))
    
    # full_text = [[[t] for t in text] for text in full_text]
    self.full_text = full_text
    # print(full_text)

    # Read labels from file
    with open(self.label_path) as label_reader:
      self.full_labels = np.loadtxt(label_reader)
    
    print(len(self.full_labels)-1)
    self.full_images = []

    for idx in range(len(self.full_labels)-1):
      image = cv2.imread(self.rawimage_path+ str(int(idx)) + ".jpg")
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
      while image.shape[0]>200 or image.shape[1]>200:
        image = cv2.resize(image, (int(0.9* image.shape[0]), int(0.9* image.shape[1])))

      image = np.expand_dims(image, axis = -1)
      image = np.asarray(image, dtype = 'float32')
      new_image = np.zeros((200 - image.shape[0], image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 0)
      new_image = np.zeros((image.shape[0], 200-image.shape[1], image.shape[2]), dtype='float32')
      image = np.append(image, new_image, axis = 1)
      self.full_images.append(np.asarray(image))

    
  def __len__(self):

    # In this dataset we assume each utterance as a sample
    # Get the number of utterances

    self.len = len(self.full_labels)-1
    return self.len

  def __getitem__(self, idx):
    
    self.image = self.full_images[idx]
    self.text = self.full_text[int(idx)]

    label = np.zeros((3))
    label[int(self.full_labels[idx])] = 1
    self.label = label

    return {'image': self.image, 'text': self.text, 'ID': idx, 'label':self.label}



In [76]:
# # Now the dataset is ready to use with data loader 

text_path = "/content/english_train.txt"
image_path = "/content/image_index_train.txt"
label_path = "/content/sentiment_train.txt"
rawimage_path = "/content/train_ende/"

train_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle = False)

text_path = "/content/english_dev.txt"
image_path = "/content/image_index_dev.txt"
label_path = "/content/sentiment_dev.txt"
rawimage_path = "/content/dev/"

dev_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
dev_data_loader = DataLoader(dev_dataset, batch_size=32, shuffle = False)


In [3]:
import torch
class ModelImage(torch.nn.Module):
	def __init__(self):
		super().__init__()
		self.model = torch.nn.Sequential(
			#Input = 3 x 32 x 32, Output = 32 x 32 x 32
			torch.nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			#Input = 32 x 32 x 32, Output = 32 x 16 x 16
			torch.nn.MaxPool2d(kernel_size=2),

			#Input = 32 x 16 x 16, Output = 64 x 16 x 16
			torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			#Input = 64 x 16 x 16, Output = 64 x 8 x 8
			torch.nn.MaxPool2d(kernel_size=3),

      densenet.features.denseblock4.denselayer16.conv2,
			torch.nn.ReLU(),
			torch.nn.MaxPool2d(kernel_size=3),

			torch.nn.Flatten(),
			torch.nn.Linear(3872 , 32),
			torch.nn.ReLU(),
      torch.nn.Linear(32 , 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 3)
		)
    

	def forward(self, x):
		return self.model(x)


In [6]:
device = 'cuda'
# device = 'cpu'

print(torch.cuda.is_available())
model_image = ModelImage().to(device)

model_image

True


ModelImage(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=3872, out_features=32, bias=True)
    (11): ReLU()
    (12): Linear(in_features=32, out_features=32, bias=True)
    (13): ReLU()
    (14): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [79]:

for param in model_image.model[6].parameters():
  param.requires_grad_ = False

epoch_num = 5
lr = 0.0001

cross_entropy = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_image.parameters(), lr = lr)

train_losses = []
image_dev_losses = []

for epoch in range(epoch_num):

  print("epoch ", epoch, "/", epoch_num, end = ": ")
  train_loss = 0
  dev_loss = 0

  model_image.train()
  
  for i, batch in enumerate(train_data_loader):

    # print(i, end = " ")
    images = torch.permute(batch['image'], (0, 3, 2, 1)).to(device)
    labels =  batch['label'].to(device)
    
    outputs = torch.softmax(model_image(images), 1)
    loss = cross_entropy(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    # print(i, end = " ")
  train_losses.append(train_loss/len(train_data_loader))
  torch.save(model_image.state_dict(), "model3_" + str(epoch))
  for i, batch in enumerate(dev_data_loader):

    images = torch.permute(batch['image'].to(device), (0, 3, 2, 1))
    labels =  batch['label'].to(device)

    outputs = torch.softmax(model_image(images), 1)
    loss = cross_entropy(outputs, labels)
    dev_loss += loss.item()

  image_dev_losses.append(dev_loss/len(dev_data_loader))

  print("\ntrain loss: ", train_losses[-1], ", dev loss: ", image_dev_losses[-1])




epoch  0 / 5: 
train loss:  1.0936529239622965 , dev loss:  1.1051850757391761
epoch  1 / 5: 
train loss:  1.086620802165702 , dev loss:  1.1038341353212011
epoch  2 / 5: 
train loss:  1.0843838366318401 , dev loss:  1.1012993846205796
epoch  3 / 5: 
train loss:  1.0821227915451646 , dev loss:  1.0956162835043743
epoch  4 / 5: 
train loss:  1.0775522862194697 , dev loss:  1.104856388979207


In [10]:
index = np.argmin(image_dev_losses)
model_image = ModelImage().to(device)
model_image.load_state_dict(torch.load("model3_" + str(index)))
model_image.eval()

ModelImage(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=3872, out_features=32, bias=True)
    (11): ReLU()
    (12): Linear(in_features=32, out_features=32, bias=True)
    (13): ReLU()
    (14): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [11]:
text_path = "/content/english_test.txt"
image_path = "/content/image_index_test.txt"
label_path = "/content/sentiment_test.txt"
rawimage_path = "/content/test/"

test_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle = False)


outputs = []
true_output = []
i = 0
with torch.no_grad():
  for batch in test_data_loader:
    images = batch['image']
    labels = batch['label']

    i += 1
    for image, label in zip(images, labels): 

      image = np.expand_dims(image.numpy(), axis = 0)
      output = model_image(torch.permute(torch.Tensor(image), (0, 3, 2, 1)).to('cuda'))
      output = torch.softmax(output, dim=1)
      outputs.append(np.argmax(output.to('cpu').numpy()))
      true_output.append(np.argmax(label.numpy()))

from torchmetrics import Accuracy
true_output = torch.tensor(np.asarray(true_output))
outputs = torch.tensor(np.asarray(outputs))
Accuracy(task="multiclass", num_classes=3)(true_output, outputs)


tensor(0.4128)

In [12]:
# # Now the dataset is ready to use with data loader 

text_path = "/content/english_train.txt"
image_path = "/content/image_index_train.txt"
label_path = "/content/sentiment_train.txt"
rawimage_path = "/content/train_ende/"

train_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle = False)

text_path = "/content/english_dev.txt"
image_path = "/content/image_index_dev.txt"
label_path = "/content/sentiment_dev.txt"
rawimage_path = "/content/dev/"

dev_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
dev_data_loader = DataLoader(dev_dataset, batch_size=32, shuffle = False)


20239
5062


In [53]:
def extract_face(image):

  face_cascade  = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")
  image = np.asarray(image, dtype = 'uint8')
  face_locations = face_cascade.detectMultiScale(image, 1.1, 4)

  face_images = []
  for location in face_locations:

    face = np.asarray(image[location[1]:location[1] + location[3], location[0]:location[0] + location[2]])


    while face.shape[0]>100 or face.shape[1]>100:
      face = cv2.resize(face, (int(0.5* face.shape[0]), int(0.5*face.shape[1])))

    
    face = cv2.resize(face, (int(int(100/face.shape[0])* face.shape[0]), (int(100/face.shape[1]) * face.shape[1])))
    
    face =  np.expand_dims(face, axis = -1)
    print(face.shape)

    new_face = np.zeros((100 - face.shape[0], face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 0)
    new_face = np.zeros((face.shape[0], 100-face.shape[1], face.shape[2]), dtype='float32')
    face = np.append(face, new_face, axis = 1)

    face_images.append(face)

  return np.asarray(face_images, dtype = 'float32')


In [55]:
# Part 3 

device = 'cuda'
# index = np.argmin(dev_losses)
index = 0
face_model = CNN().to(device)
face_model.load_state_dict(torch.load("model2_" + str(index)))
face_model.eval()

# index = np.argmin(image_dev_losses)
index = 3
model_image = ModelImage().to(device)
model_image.load_state_dict(torch.load("model3_" + str(index)))
model_image.eval()



text_path = "/content/english_test.txt"
image_path = "/content/image_index_test.txt"
label_path = "/content/sentiment_test.txt"
rawimage_path = "/content/test/"

test_dataset = SentimentDataset(text_path, image_path, label_path, rawimage_path)
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle = False)


outputs = []
true_output = []
i = 0
with torch.no_grad():
  for batch in test_data_loader:
    images = batch['image']
    labels = batch['label']
    # if i == 1000:
    #   break
    i += 1
    for image, label in zip(images, labels): 

      true_output.append(np.argmax(label.numpy()))


      image = np.expand_dims(image.numpy(), axis = 0)
      output1 = model_image(torch.permute(torch.Tensor(image), (0, 3, 2, 1)).to('cuda'))
      output1 = torch.softmax(output, dim=1)
      # Total image result
      res1 = np.argmax(output1.to('cpu').numpy())

      temp_output = []
      for face in extract_face(image):

        face = np.asarray([face])
        output = face_model(torch.permute(torch.Tensor(face), (0, 3, 2, 1)))
        output = torch.softmax(output, dim = 1)
        temp_output.append(torch.argmax(output).to('cpu'))

      if len(temp_output)==0:
        temp_output.append(1)

      u, c = np.unique(np.asarray(temp_output), return_counts = True)

      # Face recognition result
      res2 = u[np.argmax(c)]

      # Number of faces
      res3 = len(temp_output)

      if res3 == 0:
        outputs.append(res1)

      else:
        outputs.append(res2)


from torchmetrics import Accuracy
true_output = torch.tensor(np.asarray(true_output))
outputs = torch.tensor(np.asarray(outputs))
Accuracy(task="multiclass", num_classes=3)(true_output, outputs)


5066


tensor(0.4270)